## **Detecting and Monitoring Land Use Transformation in Córdoba**

### Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [2]:
import ee

# Initialize GEE
ee.Authenticate()
# Initialize GEE with your GCP project ID
ee.Initialize(project='gee-land-use-cordoba')

In [3]:
# Load a sample dataset
dem = ee.Image('USGS/SRTMGL1_003')

# Print metadata
print(dem.getInfo())

{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [1296001, 417601], 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -180.0001388888889, 0, -0.0002777777777777778, 60.00013888888889]}], 'version': 1641990767055141, 'id': 'USGS/SRTMGL1_003', 'properties': {'system:visualization_0_min': '0.0', 'type_name': 'Image', 'keywords': ['dem', 'elevation', 'geophysical', 'nasa', 'srtm', 'topography', 'usgs'], 'thumb': 'https://mw1.google.com/ges/dd/images/SRTM90_V4_thumb.png', 'description': '<p>The Shuttle Radar Topography Mission (SRTM, see <a href="https://onlinelibrary.wiley.com/doi/10.1029/2005RG000183/full">Farr\net al. 2007</a>)\ndigital elevation data is an international research effort that\nobtained digital elevation models on a near-global scale. This\nSRTM V3 product (SRTM Plus) is provided by NASA JPL\nat a resolution of 1 arc-second (approximately 30m).</p><p>This dataset

In [4]:
import geemap

# Define AoIs with provided coordinates
aoi_1 = ee.Geometry.Rectangle([-64.53518510984365, -32.09793491720308, -64.36758169478006, -31.98599080824592])  # Calmayo
aoi_2 = ee.Geometry.Rectangle([-64.19504913677835, -30.67255559541076, -63.92899759153784, -30.43974550819874])  # Las Peñas
aoi_3 = ee.Geometry.Rectangle([-64.8501456212574, -32.06252532553296, -64.70773302600543, -31.93373503424498])  # Villa Alpina

# Create an interactive map
Map = geemap.Map()

# Add AoIs to the map
Map.addLayer(aoi_1, {'color': 'red'}, 'Calmayo')
Map.addLayer(aoi_2, {'color': 'blue'}, 'Las Peñas')
Map.addLayer(aoi_3, {'color': 'green'}, 'Villa Alpina')

# Center the map on Córdoba
Map.centerObject(aoi_1, 10)
Map

Map(center=[-32.041979015931354, -64.45138340231044], controls=(WidgetControl(options=['position', 'transparen…

#### Load and Preprocess Satellite Imagery

In [5]:
def load_sentinel2(start_date, end_date, region):
    return (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterDate(start_date, end_date)
        .filterBounds(region)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))  # Max 30% cloud coverage
        .select(['B4', 'B3', 'B2', 'B8', 'B11'])  # Select necessary bands before applying media
    )

#### Load Images for Summer and Winter

In [6]:
def load_landsat8(start_date, end_date, region):
    return (
        ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
        .filterDate(start_date, end_date)
        .filterBounds(region)
        .filter(ee.Filter.lt('CLOUD_COVER', 30))  # Max 30% cloud coverage
        .select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])  # Updated band names for Collection 2, Level 2
    )

In [7]:
# Load Sentinel-2 images for all AoIs and periods (3-year timeframe)
summer_sentinel2_aoi1 = load_sentinel2('2017-12-01', '2020-03-31', aoi_1)
winter_sentinel2_aoi1 = load_sentinel2('2017-06-01', '2019-09-30', aoi_1)

summer_sentinel2_aoi2 = load_sentinel2('2017-12-01', '2020-03-31', aoi_2)
winter_sentinel2_aoi2 = load_sentinel2('2017-06-01', '2019-09-30', aoi_2)

summer_sentinel2_aoi3 = load_sentinel2('2017-12-01', '2020-03-31', aoi_3)
winter_sentinel2_aoi3 = load_sentinel2('2017-06-01', '2019-09-30', aoi_3)

# Load Landsat 8 images for all AoIs and periods (3-year timeframe)
summer_landsat8_aoi1 = load_landsat8('2013-12-01', '2016-03-31', aoi_1)
winter_landsat8_aoi1 = load_landsat8('2013-06-01', '2015-09-30', aoi_1)

summer_landsat8_aoi2 = load_landsat8('2013-12-01', '2016-03-31', aoi_2)
winter_landsat8_aoi2 = load_landsat8('2013-06-01', '2015-09-30', aoi_2)

summer_landsat8_aoi3 = load_landsat8('2013-12-01', '2016-03-31', aoi_3)
winter_landsat8_aoi3 = load_landsat8('2013-06-01', '2015-09-30', aoi_3)

In [8]:
# Store Sentinel-2 collections in a list for easier access
sentinel2_collections = [
    (summer_sentinel2_aoi1, winter_sentinel2_aoi1),
    (summer_sentinel2_aoi2, winter_sentinel2_aoi2),
    (summer_sentinel2_aoi3, winter_sentinel2_aoi3),
]

# Store Landsat 8 collections in a list for easier access
landsat8_collections = [
    (summer_landsat8_aoi1, winter_landsat8_aoi1),
    (summer_landsat8_aoi2, winter_landsat8_aoi2),
    (summer_landsat8_aoi3, winter_landsat8_aoi3),
]

In [9]:
# Example for Sentinel-2 collections
for i, (summer_collection, winter_collection) in enumerate(sentinel2_collections):
    aoi_name = f"aoi_{i + 1}"

    # Check summer collection
    summer_size = summer_collection.size().getInfo()  # Requires ee.Initialize()
    print(f"Sentinel-2 Summer ({aoi_name}): {summer_size} images found.")
    if summer_size > 0:
        summer_collection = summer_collection.median()
    else:
        print(f"Warning: No summer images for {aoi_name}!")

    # Check winter collection
    winter_size = winter_collection.size().getInfo()
    print(f"Sentinel-2 Winter ({aoi_name}): {winter_size} images found.")
    if winter_size > 0:
        winter_collection = winter_collection.median()
    else:
        print(f"Warning: No winter images for {aoi_name}!")

Sentinel-2 Summer (aoi_1): 100 images found.
Sentinel-2 Winter (aoi_1): 60 images found.
Sentinel-2 Summer (aoi_2): 185 images found.
Sentinel-2 Winter (aoi_2): 105 images found.
Sentinel-2 Summer (aoi_3): 100 images found.
Sentinel-2 Winter (aoi_3): 60 images found.


In [10]:
# Check Landsat 8 collections
for i, (summer_collection, winter_collection) in enumerate(landsat8_collections):
    aoi_name = f"aoi_{i + 1}"

    # Process summer collection
    summer_size = summer_collection.size().getInfo()
    print(f"Landsat 8 Summer ({aoi_name}): {summer_size} images found.")
    if summer_size > 0:
        summer_collection = summer_collection.median()
    else:
        print(f"Warning: Summer Landsat 8 collection is empty for {aoi_name}.")

    # Process winter collection
    winter_size = winter_collection.size().getInfo()
    print(f"Landsat 8 Winter ({aoi_name}): {winter_size} images found.")
    if winter_size > 0:
        winter_collection = winter_collection.median()
    else:
        print(f"Warning: Winter Landsat 8 collection is empty for {aoi_name}.")

Landsat 8 Summer (aoi_1): 22 images found.
Landsat 8 Winter (aoi_1): 21 images found.
Landsat 8 Summer (aoi_2): 21 images found.
Landsat 8 Winter (aoi_2): 19 images found.
Landsat 8 Summer (aoi_3): 51 images found.
Landsat 8 Winter (aoi_3): 51 images found.


In [11]:
print(type(summer_sentinel2_aoi1).__name__)  # Should print 'ImageCollection'
print(type(winter_sentinel2_aoi1).__name__)  # Should print 'ImageCollection'

ImageCollection
ImageCollection


### Data Preprocessing (Data Harmonization)

**Sentinel-2 Bands**
* B2: Blue (10m)

* B3: Green (10m)

* B4: Red (10m)

* B8: Near Infrared (NIR, 10m)

* B11: Shortwave Infrared (SWIR, 20m)

**Landsat 8 Bands**
* SR_B2: Blue (30m)

* SR_B3: Green (30m)

* SR_B4: Red (30m)

* SR_B5: Near Infrared (NIR, 30m)

* SR_B6: Shortwave Infrared 1 (SWIR1, 30m)

* SR_B7: Shortwave Infrared 2 (SWIR2, 30m)

Combining Sentinel-2 and Landsat 8 data into a single dataset for each AOI


**Including Additional Indices**

**Common Indices**:

NDWI (Normalized Difference Water Index): For water body detection.

* Sentinel-2: (B3 - B8) / (B3 + B8)

* Landsat 8: (SR_B3 - SR_B5) / (SR_B3 + SR_B5)

NDBI (Normalized Difference Built-Up Index): For urban area detection.

* Sentinel-2: (B11 - B8) / (B11 + B8)

* Landsat 8: (SR_B6 - SR_B5) / (SR_B6 + SR_B5)

EVI (Enhanced Vegetation Index): For vegetation monitoring.

* Sentinel-2: 2.5 * (B8 - B4) / (B8 + 6 * B4 - 7.5 * B2 + 1)

* Landsat 8: 2.5 * (SR_B5 - SR_B4) / (SR_B5 + 6 * SR_B4 - 7.5 * SR_B2 + 1)



##### Processing Sentinel-2 Data for All AOIs

Computing medians and add spectral indices (NDVI, NDWI, NDBI, EVI) for Sentinel-2 data

In [12]:
def process_s2(summer_collection, winter_collection):
    # Compute medians
    summer_median = summer_collection.median() if summer_collection.size().getInfo() > 0 else None
    winter_median = winter_collection.median() if winter_collection.size().getInfo() > 0 else None

    # Add indices for summer median
    if summer_median:
        summer_median = summer_median.addBands([
            summer_median.normalizedDifference(['B8', 'B4']).rename('NDVI'),  # NDVI
            summer_median.normalizedDifference(['B3', 'B8']).rename('NDWI'),  # NDWI
            summer_median.normalizedDifference(['B11', 'B8']).rename('NDBI'),  # NDBI
            summer_median.expression(
                '2.5 * (NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1)', {
                    'NIR': summer_median.select('B8'),
                    'RED': summer_median.select('B4'),
                    'BLUE': summer_median.select('B2')
                }
            ).rename('EVI')  # EVI
        ])

    # Add indices for winter median
    if winter_median:
        winter_median = winter_median.addBands([
            winter_median.normalizedDifference(['B8', 'B4']).rename('NDVI'),  # NDVI
            winter_median.normalizedDifference(['B3', 'B8']).rename('NDWI'),  # NDWI
            winter_median.normalizedDifference(['B11', 'B8']).rename('NDBI'),  # NDBI
            winter_median.expression(
                '2.5 * (NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1)', {
                    'NIR': winter_median.select('B8'),
                    'RED': winter_median.select('B4'),
                    'BLUE': winter_median.select('B2')
                }
            ).rename('EVI')  # EVI
        ])

    return summer_median, winter_median

##### Processing Landsat 8 Data for All AOIs

Compute medians and add spectral indices (NDVI, NDWI, NDBI, EVI) for Landsat 8 data.

In [13]:
def process_l8(summer_collection, winter_collection):
    # Compute medians
    summer_median = summer_collection.median() if summer_collection.size().getInfo() > 0 else None
    winter_median = winter_collection.median() if winter_collection.size().getInfo() > 0 else None

    # Add indices for summer median
    if summer_median:
        summer_median = summer_median.addBands([
            summer_median.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI'),  # NDVI
            summer_median.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI'),  # NDWI
            summer_median.normalizedDifference(['SR_B6', 'SR_B5']).rename('NDBI'),  # NDBI
            summer_median.expression(
                '2.5 * (NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1)', {
                    'NIR': summer_median.select('SR_B5'),
                    'RED': summer_median.select('SR_B4'),
                    'BLUE': summer_median.select('SR_B2')
                }
            ).rename('EVI')  # EVI
        ])

    # Add indices for winter median
    if winter_median:
        winter_median = winter_median.addBands([
            winter_median.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI'),  # NDVI
            winter_median.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI'),  # NDWI
            winter_median.normalizedDifference(['SR_B6', 'SR_B5']).rename('NDBI'),  # NDBI
            winter_median.expression(
                '2.5 * (NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1)', {
                    'NIR': winter_median.select('SR_B5'),
                    'RED': winter_median.select('SR_B4'),
                    'BLUE': winter_median.select('SR_B2')
                }
            ).rename('EVI')  # EVI
        ])

    return summer_median, winter_median

In [14]:
def fuse_data(s2_img, l8_img):
    # Combine all bands and indices
    return ee.Image.cat([
        s2_img.select(['B2', 'B3', 'B4', 'B8', 'B11']),  # Sentinel-2 bands
        s2_img.select(['NDVI', 'NDWI', 'NDBI', 'EVI']),  # Sentinel-2 indices
        l8_img.select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']),  # Landsat 8 bands
        l8_img.select(['NDVI', 'NDWI', 'NDBI', 'EVI'])  # Landsat 8 indices
    ]).rename([
        'S2_BLUE', 'S2_GREEN', 'S2_RED', 'S2_NIR', 'S2_SWIR',  # Sentinel-2 bands
        'S2_NDVI', 'S2_NDWI', 'S2_NDBI', 'S2_EVI',  # Sentinel-2 indices
        'L8_BLUE', 'L8_GREEN', 'L8_RED', 'L8_NIR', 'L8_SWIR1', 'L8_SWIR2',  # Landsat 8 bands
        'L8_NDVI', 'L8_NDWI', 'L8_NDBI', 'L8_EVI'  # Landsat 8 indices
    ])

In [15]:
# Process and update the variables with the median images
summer_s2_aoi1, winter_s2_aoi1 = process_s2(summer_sentinel2_aoi1, winter_sentinel2_aoi1)
summer_s2_aoi2, winter_s2_aoi2 = process_s2(summer_sentinel2_aoi2, winter_sentinel2_aoi2)
summer_s2_aoi3, winter_s2_aoi3 = process_s2(summer_sentinel2_aoi3, winter_sentinel2_aoi3)

summer_l8_aoi1, winter_l8_aoi1 = process_l8(summer_landsat8_aoi1, winter_landsat8_aoi1)
summer_l8_aoi2, winter_l8_aoi2 = process_l8(summer_landsat8_aoi2, winter_landsat8_aoi2)
summer_l8_aoi3, winter_l8_aoi3 = process_l8(summer_landsat8_aoi3, winter_landsat8_aoi3)

Fuse Data for All AOIs

In [16]:
# Fuse data for AOI 1
combined_aoi1 = fuse_data(summer_s2_aoi1, summer_l8_aoi1)
# Fuse data for AOI 2
combined_aoi2 = fuse_data(summer_s2_aoi2, summer_l8_aoi2)
# Fuse data for AOI 3
combined_aoi3 = fuse_data(summer_s2_aoi3, summer_l8_aoi3)

In [17]:
# Verify the Fused Data
print(combined_aoi1.bandNames().getInfo())
print(combined_aoi2.bandNames().getInfo())
print(combined_aoi3.bandNames().getInfo())

['S2_BLUE', 'S2_GREEN', 'S2_RED', 'S2_NIR', 'S2_SWIR', 'S2_NDVI', 'S2_NDWI', 'S2_NDBI', 'S2_EVI', 'L8_BLUE', 'L8_GREEN', 'L8_RED', 'L8_NIR', 'L8_SWIR1', 'L8_SWIR2', 'L8_NDVI', 'L8_NDWI', 'L8_NDBI', 'L8_EVI']
['S2_BLUE', 'S2_GREEN', 'S2_RED', 'S2_NIR', 'S2_SWIR', 'S2_NDVI', 'S2_NDWI', 'S2_NDBI', 'S2_EVI', 'L8_BLUE', 'L8_GREEN', 'L8_RED', 'L8_NIR', 'L8_SWIR1', 'L8_SWIR2', 'L8_NDVI', 'L8_NDWI', 'L8_NDBI', 'L8_EVI']
['S2_BLUE', 'S2_GREEN', 'S2_RED', 'S2_NIR', 'S2_SWIR', 'S2_NDVI', 'S2_NDWI', 'S2_NDBI', 'S2_EVI', 'L8_BLUE', 'L8_GREEN', 'L8_RED', 'L8_NIR', 'L8_SWIR1', 'L8_SWIR2', 'L8_NDVI', 'L8_NDWI', 'L8_NDBI', 'L8_EVI']
